In [71]:
import text_utilities as tu
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
wordset = tu.from_path('../data/experiment/verbs')

verb_dict = tu.count_syllables(wordset)
verb_dict.keys()
for i in range(1,9): # von mächtigster menge absteigend morpheme analysieren
    print(i, len(verb_dict[i]))

1 5010
2 27261
3 30123
4 14810
5 4455
6 1005
7 193
8 32


Tokenizer-Vokabular anhand des Token-Vokabulars erstellen:

In [185]:
"""mapping 1010101 comparing words left and right"""
# durch key
maps = []
target = "beleidigst"
l = len(target)
cnt = 0
cc1=0
cc2=0
cc3=0
#for v in verb_dict[tu.count_syllables(target)]:
for k in verb_dict:
    for v in verb_dict[k]:
        pair = (v, target)
        case = tu.match_ends(v, target)
        smaller = min(pair, key=len)
        longer = max(pair, key=len)
        diff = len(longer) - len(smaller)


        if case.get("any"):
            cnt+=1
            if diff:

                if case.get("first"):
                    cc1+=1
                    m = list(map(tu.char_compare, smaller, longer))
                    while len(m) < l:
                        m.append(0) # padding
                    maps.append(m)


                if case.get("last"):
                    m = []
                    cc2+=1
                    for i in range(len(smaller)):
                        m.append(1 if smaller[i] == longer[i+diff] else 0)
                    while len(m) < l:
                        m.insert(0, 0) # padding
                    maps.append(m)


            else:
                cc3+=1
                m = list(map(tu.char_compare, pair[0], pair[1]))
                maps.append(m)

print(cc1, cc2, cc3)

7035 20684 4347


In [186]:
# Summe aller gleichlangen maps untereinander, bildet nur Tendenz ab
#itertools recipes
new_maps = [x for x in maps if len(x)==len(target)]
sum_new_maps = [sum(x) for x in zip(*new_maps)]
sum_new_maps

[8357, 7390, 1155, 4654, 2079, 846, 3985, 673, 7971, 24475]

In [187]:
# konvertiere map in string, nur vorübergehend?
str_maps = []
for m in maps:
    mstr = ""
    for i in m:
        mstr += str(i)
    str_maps.append(mstr)
print(str_maps[:10], len(str_maps))

['1101000000', '0000001001', '0000000001', '0001000001', '0000000011', '0000000001', '0000101001', '0001000001', '0000000001', '0000000001'] 32066


In [188]:
import collections as cl
n_maps = cl.Counter(str_maps).most_common(50)
#n_maps[:50]

In [189]:
# neue maps ignorieren unterschiede im wortinneren
import regex as rex
recount_map = []
for cnt in str_maps:
    sr = rex.sub(pattern='([^^1][0*1*]+[^$1])', repl=lambda m: len(m.group(1))*"0",string=cnt)
    recount_map.append(sr)

# recount_map

In [190]:
# word map after
regexed_listed = [list(i) for i in recount_map]
regexed_inted = []
for i in regexed_listed:
    regexed_inted.append([int(j) for j in i])

new_maps = [x for x in regexed_inted if len(x)==len(target)]
sum_new_maps = [sum(x) for x in zip(*new_maps)]
sum_new_maps


[8357, 4989, 275, 81, 14, 43, 179, 462, 7871, 24475]

In [36]:
n_maps = cl.Counter(str_maps).most_common(50)
n_maps[:10]

[('000000001', 1311),
 ('111000000', 972),
 ('111000010', 663),
 ('000000011', 505),
 ('000001001', 433),
 ('111100000', 310),
 ('000011001', 183),
 ('010000001', 166),
 ('000000101', 154),
 ('000010001', 126)]

In [32]:

sorted_maps = sorted(recount_map, key=lambda t: t[0])
sorted_maps

[['000000001', 15],
 ['000000011', 334],
 ['000000011', 17],
 ['000000011', 16],
 ['000000011', 8],
 ['000000111', 34],
 ['000001111', 143],
 ['000001111', 10],
 ['000001111', 9],
 ['000011111', 5],
 ['100000000', 115],
 ['100000000', 9],
 ['100000000', 7],
 ['100000011', 9],
 ['110000000', 12]]

In [44]:
def map_subword(target: str, map:str) -> str:
    """Returns a subword from a target string and a map. Yet to implement maps with 1 on both ends."""
    if map.startswith("1"):
        return target[:map.count("1")]+"##"
    elif map.endswith("1"):
        return "##"+target[-map.count("1"):]
print(map_subword("verstehen", "000000011"))

##en


In [90]:
recount_map

['1100000000',
 '0000000001',
 '0000000001',
 '0000000001',
 '0000000011',
 '0000000001',
 '0000001001',
 '0000000001',
 '0000000001',
 '0000000001',
 '0000001001',
 '1100000000',
 '0000000001',
 '0000000001',
 '0000000001',
 '1100000000',
 '1110000000',
 '1000000000',
 '1000000000',
 '0000000011',
 '0000000011',
 '0000000011',
 '0000000001',
 '0000000011',
 '0000000001',
 '0000000011',
 '1100000000',
 '0000000001',
 '1000000000',
 '0000000001',
 '0000000001',
 '0000000001',
 '0000000011',
 '0000000001',
 '0000000001',
 '1000000000',
 '0000000001',
 '0000000011',
 '1100000001',
 '1000000000',
 '0000000001',
 '0000000001',
 '1000000000',
 '1000000000',
 '0000000001',
 '0000000001',
 '1100000000',
 '0000000001',
 '1100000000',
 '1000000000',
 '0000000001',
 '1100000000',
 '0000000011',
 '1000000000',
 '0000000001',
 '0000000001',
 '1100000000',
 '0000000011',
 '0000000011',
 '0000000001',
 '0000000001',
 '0000000001',
 '0000000001',
 '0000000001',
 '0000000001',
 '0000000001',
 '00000000

In [184]:
rex.sub(pattern='([^^1][0*1*]+[^$1])', repl=lambda m: len(m.group(1))*"0",string="1111001101111010111")

'1111000000000000111'

In [182]:
len("1110010101011")
len("1110010101011")

13